In [1]:
from analytics import Analytics
import pandas as pd

calculadora =  Analytics()

In [3]:
print('-------- Rodrigo --------')
res = calculadora.calcular_patrimonio('rodrigo')
print(f"Patrimonio total: {res:,.2f}\n")

print('-------- Dayane --------')
res = calculadora.calcular_patrimonio('dayane')
print(f"Patrimonio total: {res:,.2f}\n")

-------- Rodrigo --------
Real: 51,475.35
Dolar: 2,467.94 (12,074.40)
Ativos Real: 31,927.46
Ativos Dolar: 2,081.40 (10,183.25)
Patrimonio total: 105,660.46

-------- Dayane --------
Real: 12,414.40
Dolar: 435.03 (2,128.38)
Ativos Real: 0.00
Ativos Dolar: 520.35 (2,545.81)
Patrimonio total: 17,088.60



In [4]:
p = calculadora.calcular_patrimonio_reservas(dono='rodrigo')
print(p)

p = calculadora.calcular_patrimonio_reservas(dono='dayane')
print(p)

{'reservas': 32669.3, 'caixa_investimentos': 17850.98, 'robank_dayane': 0.0}
{'reservas': 0.0, 'caixa_investimentos': 0.0, 'robank_dayane': 12414.400000000001}


In [5]:
p = calculadora.calcular_valorizacao_ativos(dono='rodrigo')
p = pd.DataFrame(p)
print(p)

p = calculadora.calcular_valorizacao_ativos(dono='dayane')
p = pd.DataFrame(p)
print(p)

                       BBAS3        CMIG4         ITSA4       MGLU3  \
price_today        47.000000    11.990000      9.630000    3.000000   
quantidade        100.000000   200.000000   1760.000000  100.000000   
valorizacao         0.291209     0.029185      0.074681   -0.145299   
valor_investido  3640.000000  2330.000000  15771.000000  351.000000   
preco_medio        36.400000    11.650000      8.960795    3.510000   

                       TAEE3          MMM        ITUB4       IRBR3  
price_today        11.810000   104.070000    27.500000    39.02000  
quantidade        300.000000    20.000000   100.000000    33.00000  
valorizacao        -0.107332     0.017103     0.014012     0.28766  
valor_investido  3969.000000  2046.400000  2712.000000  1000.00000  
preco_medio        13.230000   102.320000    27.120000    30.30303  
                        MMM
preco_medio      102.320000
price_today      104.070000
quantidade         5.000000
valor_investido  511.600000
valorizacao        

In [6]:
p = calculadora.calcular_ativos_proventos(dono='rodrigo')
p = pd.DataFrame(p)
print(p)

p = calculadora.calcular_ativos_proventos(dono='dayane')
p = pd.DataFrame(p)
print(p)

            BBAS3  CMIG4   ITSA4  IRBR3   TAEE3  ITUB4
proventos  300.63  46.61  316.54   7.41  230.56  53.32
Empty DataFrame
Columns: []
Index: []


In [7]:
p = calculadora.calcular_cdi_proventos(dono='rodrigo')
p = pd.DataFrame(p)
print(p)

p = calculadora.calcular_cdi_proventos(dono='dayane')
p = pd.DataFrame(p)
print(p)

            reservas  caixa_investimentos
valor       32669.30             17850.98
rendimento    937.54               235.59
            robank_dayane
rendimento         234.33
valor            12414.40
